# TQDM Demo for Data Science

## Intro

In [3]:
from tqdm import tqdm

[TQDM](https://github.com/noamraph/tqdm) is a great tool if you ever wanted to quickly create a progress bar for your loops. For Data Science this is expecially useful when you're applying a transformation and you want to know if you have time to get another beer.

## How to use

From the TQDM documentation:

> **just wrap any iterator with "tqdm(iterator)"**, and you're done!

Yep, it's that simple!

Let's look at what this progress bar looks like:

In [14]:
# Iterate from 0 to 999, pause for 0.01s between each iteration
import time
for i in tqdm(range(1000)):
    time.sleep(0.01)

100%|██████████| 1000/1000 [00:11<00:00, 85.26it/s]


As you can see, in Jupyter Notebook tqdm creates a one-line progress bar that shows some useful info including:

- the progress bar itself
- elapsed time
- remaining time
- rate of iteration per second

## Using TQDM with Other Data Science Packages

In [18]:
# remove nulls from 
evens = []
for x in tqdm(range(100000)):
    if x%2==0:
        evens.append(x)
print evens[:50]

100%|██████████| 100000/100000 [00:00<00:00, 382181.12it/s]

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98]
